# 멋진 작사가 만들기

## Step 1. 데이터 다운로드
실습(1) 데이터 다듬기에서 Cloud shell에 심볼릭 링크로 ~/aiffel/lyricist/data를 생성하셨다면, ~/aiffel/lyricist/data/lyrics에 데이터가 있음

## Step 2. 데이터 읽어오기

In [19]:
import glob
import os, re
import numpy as np
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


## Step 3. 데이터 정제

In [20]:
# 토큰화(Tokenize) - 단어 사전
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2. 특수문자 양쪽에 공백을 넣고
    sentence = re.sub(r'[" "]+', " ", sentence) # 3. 여러개의 공백은 하나의 공백으로 바꿉니다
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
    sentence = sentence.strip() # 5. 다시 양쪽 공백을 지웁니다
    sentence = '<start> ' + sentence + ' <end>' # 6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
    return sentence

In [21]:
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue

    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

In [22]:
def tokenize(corpus):
    # 12000 단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 15000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다. -문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=15) # maxlen=15 → 최대 15개 이하로
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...   43    3    0]
 ...
 [   5   22    9 ...   10 1013    3]
 [  37   15 9049 ...  877  647    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f3b903abc10>


## Step 4. 평가 데이터셋 분리

In [23]:
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0]
[  50    5   91  297   65   57    9  969 6042    3    0    0    0    0]


In [24]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2)

In [25]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


## Step 5. 인공지능 만들기

In [26]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [27]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256 # 워드 벡터의 차원 수
hidden_size = 1024 # 모델에 일꾼 수
lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [28]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
lyricist(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 4.32671441e-05, -5.11386606e-05, -1.25411520e-04, ...,
         -3.74097472e-05,  8.06683092e-05,  9.06248024e-05],
        [ 1.14027835e-05,  2.41030903e-05, -5.95837831e-04, ...,
          2.75464467e-04,  1.81108131e-04,  1.59710744e-05],
        [-1.08928994e-04,  1.23141272e-05, -8.60047585e-04, ...,
          4.58829862e-04,  1.72801738e-04, -1.02432059e-04],
        ...,
        [ 8.13280581e-04,  6.77825941e-04, -4.15370625e-04, ...,
          1.18397793e-03, -9.95850191e-04, -8.44660914e-04],
        [ 1.24338642e-03,  1.16056658e-03, -1.68994549e-04, ...,
          1.07568491e-03, -1.07187999e-03, -1.21123006e-03],
        [ 1.63628627e-03,  1.59013853e-03,  8.19526540e-05, ...,
          9.19577142e-04, -1.10568083e-03, -1.58773619e-03]],

       [[ 4.32671441e-05, -5.11386606e-05, -1.25411520e-04, ...,
         -3.74097472e-05,  8.06683092e-05,  9.06248024e-05],
        [-2.97605933e-04, -9.20783132e-05, -1

In [29]:
lyricist.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_3 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_1 (Dense)              multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [30]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

lyricist.compile(loss=loss, optimizer=optimizer)
lyricist.fit(dataset, epochs=10)

Epoch 1/10
686/686 [==============================] - 117s 168ms/step - loss: 3.6767
Epoch 2/10
686/686 [==============================] - 114s 167ms/step - loss: 3.1610
Epoch 3/10
686/686 [==============================] - 115s 167ms/step - loss: 2.9568
Epoch 4/10
686/686 [==============================] - 115s 167ms/step - loss: 2.8036
Epoch 5/10
686/686 [==============================] - 115s 167ms/step - loss: 2.6719
Epoch 6/10
686/686 [==============================] - 115s 167ms/step - loss: 2.5538
Epoch 7/10
686/686 [==============================] - 115s 167ms/step - loss: 2.4450
Epoch 8/10
686/686 [==============================] - 115s 168ms/step - loss: 2.3429
Epoch 9/10
686/686 [==============================] - 115s 168ms/step - loss: 2.2469
Epoch 10/10
686/686 [==============================] - 115s 168ms/step - loss: 2.1559


In [31]:
def generate_text(lyricist, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다 
    while True:
        # 1. 입력받은 문장의 텐서를 입력합니다
        predict = lyricist(test_tensor) 
        #  2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3. 2에서 예측된 word index를 문장 뒤에 붙입니다
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        #  4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
        
    return generated

In [32]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

In [33]:
generate_text(lyricist, tokenizer, init_sentence="<start> she", max_len=20)

'<start> she s got me runnin round and round <end> '

In [34]:
generate_text(lyricist, tokenizer, init_sentence="<start> so", max_len=20)

'<start> so i can see it through the lens of my eyes <end> '

## 후기


학습 데이터 개수가 제시된 124960 보다 크게 나와 코딩에 잘 못된 곳이 있는지 다른 분들과 의견 나눠 봄 <br>
아직도 주어진 데이터 처리만으로도 벅참.